# datetime
datetime是Python处理日期和时间的标准库。

## 获取当前日期和时间
我们先看如何获取当前日期和时间：

In [2]:
from datetime import datetime
now = datetime.now()
print(now)
print(type(now))

2022-08-16 09:37:48.674388
<class 'datetime.datetime'>


注意到`datetime`是模块，`datetime`模块还包含一个`datetime`类，通过`from datetime import datetime`导入的才是`datetime`这个类。

如果仅导入`import datetime`，则必须引用全名`datetime.datetime`。

`datetime.now()`返回当前日期和时间，其类型是`datetime`。

## 获取指定日期和时间
要指定某个日期和时间，我们直接用参数构造一个datetime：

In [3]:
dt = datetime(2015,4,19,12,20)# 用指定日期时间创建datetime
print(dt)

2015-04-19 12:20:00


## datetime转换为timestamp
在计算机中，时间实际上是用数字表示的。我们把1970年1月1日 00:00:00 UTC+00:00时区的时刻称为`epoch time`，记为0（1970年以前的时间timestamp为负数），当前时间就是相对于`epoch time`的秒数，称为`timestamp`。

你可以认为：

`timestamp = 0 = 1970-1-1 00:00:00 UTC+0:00`
对应的北京时间是：

`timestamp = 0 = 1970-1-1 08:00:00 UTC+8:00`

可见`timestamp`的值与时区毫无关系，因为`timestamp`一旦确定，其`UTC`时间就确定了，转换到任意时区的时间也是完全确定的，这就是为什么计算机存储的当前时间是以`timestamp`表示的，因为全球各地的计算机在任意时刻的`timestamp`都是完全相同的（假定时间已校准）。

把一个`datetime`类型转换为`timestamp`只需要简单调用`timestamp()`方法：

In [4]:
dt.timestamp()

1429417200.0

注意Python的`timestamp`是一个浮点数。如果有小数位，小数位表示毫秒数。

某些编程语言（如Java和JavaScript）的`timestamp`使用整数表示毫秒数，这种情况下只需要把`timestamp`除以1000就得到Python的浮点表示方法。

## timestamp转换为datetime

要把`timestamp`转换为`datetime`，使用`datetime`提供的`fromtimestamp()`方法：

In [5]:
t=1429417200.0
tp = datetime.fromtimestamp(t)
print(tp)

2015-04-19 12:20:00


注意到`timestamp`是一个浮点数，它没有时区的概念，而`datetime`是有时区的。上述转换是在`timestamp`和本地时间做转换。

本地时间是指当前操作系统设定的时区。例如北京时区是东8区，则本地时间：
`2015-04-19 12:20:00`

实际上就是UTC+8:00时区的时间：
`2015-04-19 12:20:00 UTC+8:00`

而此刻的格林威治标准时间与北京时间差了8小时，也就是UTC+0:00时区的时间应该是：

`2015-04-19 04:20:00 UTC+0:00`
timestamp也可以直接被转换到UTC标准时区的时间：


In [6]:
t = 1429417200.0
print(datetime.fromtimestamp(t))
print(datetime.utcfromtimestamp(t))

2015-04-19 12:20:00
2015-04-19 04:20:00


## str转换为datetime
很多时候，用户输入的日期和时间是字符串，要处理日期和时间，首先必须把`str`转换为`datetime`。转换方法是通过`datetime.strptime()`实现，需要一个日期和时间的格式化字符串：

In [7]:
strtime = '2015-6-1 18:29:59'
timeformat = '%Y-%m-%d %H:%M:%S'
cday = datetime.strptime(strtime, timeformat)
print(cday)

2015-06-01 18:29:59


字符串`'%Y-%m-%d %H:%M:%S'`规定了日期和时间部分的格式。详细的说明请参考[Python](https://docs.python.org/3/library/datetime.html#strftime-strptime-behavior)文档。

注意转换后的`datetime`是没有时区信息的。

## datetime转换为str
如果已经有了`datetime`对象，要把它格式化为字符串显示给用户，就需要转换为`str`，转换方法是通过`strftime()`实现的，同样需要一个日期和时间的格式化字符串：

In [8]:
print(datetime.now().strftime('%a,%b %d %H:%M'))

Tue,Aug 16 09:50


## datetime加减
对日期和时间进行加减实际上就是把`datetime`往后或往前计算，得到新的`datetime`。加减可以直接用+和-运算符，不过需要导入`timedelta`这个类：

In [10]:
from datetime import timedelta

now = datetime.now()
print(now)

time = now + timedelta(hours=10)
print(time)

time = now - timedelta(days=1)
print(time)

time = now + timedelta(days=2,hours=12)
print(time)

2022-08-16 09:56:45.578493
2022-08-16 19:56:45.578493
2022-08-15 09:56:45.578493
2022-08-18 21:56:45.578493


可见，使用`timedelta`你可以很容易地算出前几天和后几天的时刻。

## 本地时间转换为UTC时间
本地时间是指系统设定时区的时间，例如北京时间是`UTC+8:00`时区的时间，而`UTC`时间指`UTC+0:00`时区的时间。

一个`datetime`类型有一个时区属性`tzinfo`，但是默认为`None`，所以无法区分这个`datetime`到底是哪个时区，除非强行给`datetime`设置一个时区：

In [12]:
from datetime import timezone

tz_utc_8 = timezone(timedelta(hours=8))# 创建时区UTC+8:00

now = datetime.now()
print(now)

dt = now.replace(tzinfo=tz_utc_8)# 强制设置为UTC+8:00
print(dt)

2022-08-16 10:04:19.067292
2022-08-16 10:04:19.067292+08:00


如果系统时区恰好是UTC+8:00，那么上述代码就是正确的，否则，不能强制设置为UTC+8:00时区。

## 时区转换
我们可以先通过`utcnow()`拿到当前的UTC时间，再转换为任意时区的时间：

In [14]:
# 拿到UTC时间，并强制设置时区为UTC+0:00:
utc_dt = datetime.utcnow().replace(tzinfo=timezone.utc)
print(utc_dt)

# astimezone()将转换时区为北京时间:
bj_dt = utc_dt.astimezone(timezone(timedelta(hours=8)))
print(bj_dt)

# astimezone()将转换时区为东京时间:
tokyo_dt = utc_dt.astimezone(timezone(timedelta(hours=9)))
print(tokyo_dt)

# astimezone()将bj_dt转换时区为东京时间:
tokyo_dt2 = bj_dt.astimezone(timezone(timedelta(hours=9)))
print(tokyo_dt2)

2022-08-16 02:07:39.317432+00:00
2022-08-16 10:07:39.317432+08:00
2022-08-16 11:07:39.317432+09:00
2022-08-16 11:07:39.317432+09:00


时区转换的关键在于，拿到一个`datetime`时，要获知其正确的时区，然后强制设置时区，作为基准时间。

利用带时区的`datetime`，通过`astimezone()`方法，可以转换到任意时区。

注：不是必须从`UTC+0:00`时区转换到其他时区，任何带时区的`datetime`都可以正确转换，例如上述`bj_dt`到`tokyo_dt`的转换。

小结
`datetime`表示的时间需要时区信息才能确定一个特定的时间，否则只能视为本地时间。

如果要存储`datetime`，最佳方法是将其转换为`timestamp`再存储，因为timestamp的值与时区完全无关。